![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/University_of_Tennessee_at_Chattanooga_logo.svg/1024px-University_of_Tennessee_at_Chattanooga_logo.svg.png)
<br>
![](https://i.imgur.com/MxD5e0S.png)

# MemEx v4 Tutorial
By Stephen Lawrence in October 2021

Here is a quick tutorial with examples to teach you how to start scripting your very own memory experiments. First, we will run thru the basic commands, explaining what they do and giving demonstrations. Then there is a short section on how to edit the settings for your experiment. Lastly, there is an example script to log SEU data.

# Commands

### LOG
Call `log(data,header)` to add to or create a CSV fil with the numbers/strings contained in list `data`. The columns for the table must be given in `header` and have the same length as `data`. The default location for the output CSV is `data/log.csv` although you can change the location by giving a 3rd argument. Don't input lists or dicts, try to stick to numbers, strings, and `None`. By default the log adds the date and time as the first 2 columns in the log file. If you want to turn this off, input an optional argument (e.g. `log(data,header,timestamp=False)`).

In [1]:
import memex as x
header = ['A','B','C','D','E']
data = [1,5.25,'True',None,1000]
file = 'data/tutorial/log.csv'
x.log(data,header,file)

'data/tutorial/log.csv'

### STATUS
Call `status()` to return an array of True/False values representing which devices working properly. If a device is not working properly (returns 0), all attempts to scan or read from that device will return `None` or `np.nan`.

In [1]:
import memex as x
print('A1,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,D3')
print(x.status())

A1,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,D3
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]


### FILL
Call `fill()` to fill all devices, up to the limit set by `size`, with the pattern set by `pattern`. Note that the size and pattern can be different for each group of devices. The `fill()` function returns an array the patterns written to each group of devices.

In [3]:
import memex as x
print(x.fill())
x.pattern = [0,0,0,0]
print(x.fill())

[85, 85, 85, 85]
[0, 0, 0, 0]


### HOLD
Call `hold()` to hold the devices at low voltage for a period of time. By default the function waits until the user hits ENTER to end the trial. You can also set the period of time in advance with input `t`. The holding voltage for each group of devices is set by `voltage`. The function returns a tuple of `(t,v)` where `t` is the period in seconds, and `v` is a list of holding voltages for each group.

In [4]:
import memex as x
print(x.hold(t=2))
x.voltage = [0.6,0.8,1.0,1.2]
print(x.hold())

	holding @ [0.5, 1.0, 1.5, 2.0] ...
(2, [0.5, 1.0, 1.5, 2.0])
	holding @ [0.6, 0.8, 1.0, 1.2] ...
	(press ENTER to return to nominal)
(0.9210710525512695, [0.6, 0.8, 1.0, 1.2])


### SCAN
Call `scan()` to quickly scan the memories for faults compared to the patterns set in `pattern`. The function returns a list of the faults found in each device. You can speed this up tremendously by reducing the size of the memory, at the cost of more data.

In [5]:
import memex as x
x.size = [4096,4096,4096,4096]
print(x.scan())
x.size = [1,1,1,1]
print(x.scan())

[2195456, 1667072, 2007040, 1560576, 1658880, 1691648, 1658880, 1679360, 1724416, 1437696, 1585152, nan]
[519, 516, 470, 420, 442, 493, 387, 415, 355, 333, 323, nan]


### SAVE
Call `save()` to read all the contents of the devices (up to the limit set by `size`) as an array (a list of lists). Each row in the array (e.g. `data[i]`) is a list containing the memory contents of the device. The values are decimal representations of the data bytes (as unsigned 8-bit integers), so the values range from 0 to 255. By default, the data is output to the location `data/temp/save.csv` but you should change this because it's overwritten every time the function is called. Alternatively, turn the returned data into your own pandas DataFrame and use the `df.to_csv()` method.

In [6]:
import memex as x
x.size = [1,1,1,1]
x.pattern = [0,255,85,170]
print(x.fill())
data = x.save()

import pandas as pd
df = pd.DataFrame(data,index=['A1','A2','A3','B1','B2','B3','C1','C2','C3','D1','D2','D3'])
print(df)
df.to_csv('data/tutorial/save.csv')

[0, 255, 85, 170]
     0      1      2      3      4      5      6      7      8      9     ...  \
A1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
A2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
A3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
B1  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  ...   
B2  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  ...   
B3  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  ...   
C1   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0  ...   
C2   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0  ...   
C3   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0   85.0  ...   
D1  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  ...   
D2  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  ...   
D3    NaN 

### CHECK
Call `check()` to find the location of every flipped bit in the devices (up to the limit set by `size`). The data is read from the devices and checked against `pattern`. The function returns an array similar to `save()` but the data represents an XOR between the read data and the set pattern. In other words, the data is 0 wherever the bits match and 1 wherever the bits do not match. Like with `save()`, the data bytes are represented as unsigned 8-bit integers. It is possible to extract an array of purely 1s and 0s representing each bit in the device as its own column (instead of each column representing a byte).

Like with `save()`, you could also turn the data into a pandas DataFrame

In [7]:
import memex as x
x.size = [1,1,1,1]
x.pattern = [85,85,85,85]
data = x.check()
print('data rows:',len(data))
print('data columns:',len(data[0]))

A1 = data[0]
print('A1 check:',A1[0:8])

bits = []
for i in range(len(A1)) :
    bits += [(A1[i] >> b) & 1 for b in range(8)][::-1]
print('A1 bits:', bits[0:8*8])

import pandas as pd
df = pd.DataFrame(data,index=['A1','A2','A3','B1','B2','B3','C1','C2','C3','D1','D2','D3'])
print(df)

data rows: 12
data columns: 1024
A1 check: [0, 0, 0, 0, 0, 0, 0, 0]
A1 bits: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    0     1     2     3     4     5     6     7     8     9     ...  1014  \
A1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
A2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
A3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
B1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
B2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
B3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
C1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
C2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
C3   0.0   0.0   0.0   0.0   0.0   0.

# Settings

Feel free to edit the settings below to tailor your test. You can change them in the source code `memex.py` or just set them temporarily at the start of your script.

`nominal` ~ list of four numbers (one for each group) defining the nominal voltage.

`voltage` ~ list of four numbers (one for each group) defining the holding voltage used during `hold()`.

`pattern` ~ list of four numbers (one for each group) defining the patterns used to `fill()`, `scan()`, and `check()`.

`size` ~ list of four numbers (one for each group) defining the number of kiloBytes (kB) used for all functions.

`log_file` ~ string defining the file path from the folder where `memex.py` is located to the desired command log locations.

The script below sets each of the above parameters to their default settings, and prints them to the terminal. NOTE: If you are using the command line, you should set these parameters at the same time as you run your test. Do this by separating the lines using SHIFT+ENTER instead of running each line independently with just ENTER.

In [8]:
import memex as x
x.nominal = [3.3,3.3,3.3,3.3]
x.voltage = [0.5,1.0,1.5,2.0]
x.pattern = [85,85,85,85]
x.size = [4096,4096,4096,4096]
x.log_file = 'data/log.csv'
print(x.nominal,x.voltage,x.pattern,x.size,x.log_file,sep='\n')

[3.3, 3.3, 3.3, 3.3]
[0.5, 1.0, 1.5, 2.0]
[85, 85, 85, 85]
[4096, 4096, 4096, 4096]
data/log.csv


# Scripting
Below is a simple script to fill all the devices with a pattern, hold at low voltage, wait for user to end trial, then scan for SEUs. Note that this data is only shown on the terminal and not automatically saved anywhere. A good place to start with automating experiments would be to automatically log and save these results. I suggest using pandas DataFrames or the `log()` method in the `memex` package.

In [5]:
import memex as x

# settings 
t = None # set holding time in seconds (or use None for manual operation)
x.voltage = [0.5,1.0,1.5,2.0] # set holding voltage for groups A, B, C, and D
x.pattern = [85,85,85,85] # set pattern used to fill memories (85 is default checkerboard)
x.size = [1,1,1,1] # set number of kB to use (up to 4096)
x.log_file = 'data/tutorial/log.csv' # set command log location

# run trial
print('STATUS:',x.status())
print('FILL:',x.fill())
input('\t(press ENTER to begin trial)')
print('HOLD:',x.hold(t=t))
data = x.scan()
print('SCAN:',data)
header = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','D1','D2','D3']
x.log(data,header,file='data/tutorial/data.csv')

STATUS: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
FILL: [85, 85, 85, 85]
	(press ENTER to begin trial)\
	holding @ [0.5, 1.0, 1.5, 2.0] ...
	(press ENTER to return to nominal)
HOLD: (0.6286029815673828, [0.5, 1.0, 1.5, 2.0])
SCAN: [513, 555, 434, 386, 395, 410, 301, 354, 336, 232, 238, nan]


'data/tutorial/data.csv'